#  Level Up Your Data Lakehouse 
#  with Data Source Control and 
#  Cross Collection Consistency

<img src="https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-logo-whitebackground.png" width=300/>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/Heart_coraz%C3%B3n.svg/1200px-Heart_coraz%C3%B3n.svg.png" width=100/> 
<img src="https://github.com/treeverse/lakeFS/blob/master/docs/assets/img/logo_large.png?raw=true" width=400/>


This is a companion notebook to walktrough the power of combining Delta Lake and lakeFS for you Data & ML workloads.
* This notebook has been tested with *DBR 7.3 LTS, Python 3*, Scala 2.12
* You will need a lakeFS installation  checkout -> [docs.lakefs.io](http://docs.lakefs.io/)

_🚧 This notebook may have existing environment or data requirements; it's included here so that you can see the contents and be inspired by it—but it may not run properly.🚧_

----

# What is a Data Lakehouse ?

Data Lakehouse is combining the best elements of data lakes and data warehouse into a single platform to assist data teams to operate efficiently. 
With this modern data stack and Lakehouse capabilities, we can enable multiple types of data transformations to co-exist, while eliminating the data silos in data teams. That means better data flows, simpler operational maintenance, and overall better data products!

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Lake

Optimization Layer a top blob storage for Reliability (i.e. ACID compliance) and Low Latency of Streaming + Batch data pipelines.

#What is lakeFS ?
lakeFS is an open-source project that provides a git-like version control interface for data lakes, with seamless integration to most data tools and frameworks.

lakeFS enables you to easily implement parallel pipelines for experimentation, reproducibility and CI/CD for data.

lakeFS supports AWS S3, Azure Blob Storage and Google Cloud Storage (GCS) as its underlying storage service. It is API compatible with S3 and works seamlessly with all modern data frameworks such as Spark, Hive, AWS Athena, Presto, etc.

# What is a Cross Collection Consistency ? 

Consistency between different data collections. A few examples may be:

* To join different collections in order to create a unified view of an account, a user or another entity we measure.
* To introduce the same data in different formats
* To introduce the same data with a different leading index or sorting due to performance considerations

<img src="https://lakefs.io/wp-content/uploads/2022/02/level2-git-for-data-lakefs-data-lake-1.png" width=900/>

## The Data

The data used is public data from Lending Club. It includes all funded loans from 2012 to 2017. Each loan includes applicant information provided by the applicant as well as the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. For a full view of the data please view the data dictionary available [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).


![Loan_Data](https://preview.ibb.co/d3tQ4R/Screen_Shot_2018_02_02_at_11_21_51_PM.png)

https://www.kaggle.com/wendykan/lending-club-loan-data

In [ ]:
# Configure Delta Lake Silver Path
DELTALAKE_SILVER_PATH = "lakefs://bi-reports/main/tables/loan_by_state/"

In [ ]:
%sql
SELECT * FROM delta.`lakefs://bi-reports/main/tables/loan_by_state/` LIMIT 10;

In [ ]:
%sql
SELECT * FROM delta.`lakefs://bi-reports/main/tables/loan_payments/` LIMIT 10;

In [ ]:
%sql
SELECT * FROM delta.`lakefs://bi-reports/main/tables/loan_details/` LIMIT 10;

In [ ]:
%sql 

CREATE TABLE IF NOT EXISTS loan_by_state
USING delta
LOCATION "lakefs://bi-reports/main/tables/loan_by_state/"

In [ ]:
%sql 
DESCRIBE DETAIL delta.`lakefs://bi-reports/main/tables/loan_by_state/`

In [ ]:
%sql

-- View Delta Lake table
SELECT * FROM loan_by_state

## Stop the notebook before the streaming cell, in case of a "run all"

In [ ]:
dbutils.notebook.exit("stop") 

In [ ]:
%fs ls lakefs://bi-reports/main/tables/loan_by_state/

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Unified Batch and Streaming Source and Sink

These cells showcase streaming and batch concurrent queries (inserts and reads)
* This notebook will run an `INSERT` every 10s against our `loan_stats` table
* We will run two streaming queries concurrently against this data
* Note, you can also use `writeStream` but this version is easier to run in DBCE

In [ ]:
# Read the insertion of data
loan_by_state_readStream = spark.readStream.format("delta").load(DELTALAKE_SILVER_PATH)
loan_by_state_readStream.createOrReplaceTempView("loan_by_state_readStream")

In [ ]:
%sql
select addr_state, sum(`count`) as loans from loan_by_state_readStream group by addr_state

**Wait** until the stream is up and running before executing the code below

In [ ]:
import time
i = 1
while i <= 6:
  # Execute Insert statement
  insert_sql = "INSERT INTO loan_by_state VALUES ('IA', 450)"
  spark.sql(insert_sql)
  print('loan_by_state: inserted new row of data, loop: [%s]' % i)
    
  # Loop through
  i = i + 1
  time.sleep(5)

**Note**: Once the previous cell is finished and the state of Iowa is fully populated in the map (in cell 14), click *Cancel* in Cell 14 to stop the `readStream`.

Let's review our current set of loans using our map visualization.

In [ ]:
%sql
-- Review current loans within the `loan_by_state` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state group by addr_state

Observe that the Iowa (middle state) has the largest number of loans due to the recent stream of data.  Note that the original `loan_by_state` table is updated as we're reading `loan_by_state_readStream`.

##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Full DML Support

**Note**: Full DML Support is a feature that will be coming soon to Delta Lake; the preview is currently available in Databricks.

Delta Lake supports standard DML(Data manipulation language) including UPDATE, DELETE and MERGE INTO providing developers more controls to manage their big datasets.

Let's start by creating a traditional Parquet table

In [ ]:
# Load new DataFrame based on current Delta table
lbs_df = sql("select * from loan_by_state")

# Save DataFrame to Parquet
lbs_df.write.mode("overwrite").parquet("/tmp/loan_by_state.parquet")

# Create new table on this parquet data
spark.sql("drop table if exists loan_by_state_pq")
spark.sql("create table loan_by_state_pq using parquet as select * from parquet.`/tmp/loan_by_state.parquet`")

# Review data
display(sql("select * from loan_by_state_pq"))

###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) DELETE Support

The data was originally supposed to be assigned to `WA` state, so let's `DELETE` those values assigned to `IA`

In [ ]:
%sql
-- Attempting to run `DELETE` on the Parquet table
DELETE FROM loan_by_state_pq WHERE addr_state = 'IA'

**Note**: This command fails because the `DELETE` statements are not supported in Parquet, but are supported in Delta Lake.

In [ ]:
%sql
-- Running `DELETE` on the Delta Lake table
DELETE FROM loan_by_state WHERE addr_state = 'IA'

In [ ]:
%sql
-- Review current loans within the `loan_by_state` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state group by addr_state

###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) UPDATE Support
The data was originally supposed to be assigned to `WA` state, so let's `UPDATE` those values

In [ ]:
%sql
-- Attempting to run `UPDATE` on the Parquet table
UPDATE loan_by_state_pq SET `count` = 2700 WHERE addr_state = 'WA'

**Note**: This command fails because the `UPDATE` statements are not supported in Parquet, but are supported in Delta Lake.

In [ ]:
%sql
-- Running `UPDATE` on the Delta Lake table
UPDATE loan_by_state SET `count` = 2700 WHERE addr_state = 'WA'

In [ ]:
%sql
-- Review current loans within the `loan_by_state` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state group by addr_state

###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) MERGE INTO Support

#### INSERT or UPDATE parquet: 7-step process

With a legacy data pipeline, to insert or update a table, you must:
1. Identify the new rows to be inserted
2. Identify the rows that will be replaced (i.e. updated)
3. Identify all of the rows that are not impacted by the insert or update
4. Create a new temp based on all three insert statements
5. Delete the original table (and all of those associated files)
6. "Rename" the temp table back to the original table name
7. Drop the temp table

![](https://pages.databricks.com/rs/094-YMS-629/images/merge-into-legacy.gif)


#### INSERT or UPDATE with Delta Lake

2-step process: 
1. Identify rows to insert or update
2. Use `MERGE`

In [ ]:
# Let's create a simple table to merge
items = [('IA', 10), ('CA', 2500), ('OR', None)]
cols = ['addr_state', 'count']
merge_table = spark.createDataFrame(items, cols)
merge_table.createOrReplaceTempView("merge_table")
display(merge_table)

Instead of writing separate `INSERT` and `UPDATE` statements, we can use a `MERGE` statement.

In [ ]:
%sql
MERGE INTO loan_by_state as d
USING merge_table as m
on d.addr_state = m.addr_state
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *

In [ ]:
%sql
-- Review current loans within the `loan_by_state` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state group by addr_state

##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Schema Evolution
With the `mergeSchema` option, you can evolve your Delta Lake table schema

In [ ]:
# Generate new loans with dollar amounts 
loans = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as amount from loan_by_state")
display(loans)

In [ ]:
# Let's write this data out to our Delta table
loans.write.format("delta").mode("append").save(DELTALAKE_SILVER_PATH)

**Note**: This command fails because the schema of our new data does not match the schema of our original data

In [ ]:
# Add the mergeSchema option
loans.write.option("mergeSchema","true").format("delta").mode("append").save(DELTALAKE_SILVER_PATH)

**Note**: With the `mergeSchema` option, we can merge these different schemas together.

In [ ]:
%sql
-- Review current loans within the `loan_by_state` Delta Lake table
select addr_state, sum(`amount`) as amount from loan_by_state group by addr_state order by sum(`amount`) desc limit 10

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Let's Travel back in Time!
Databricks Delta’s time travel capabilities simplify building data pipelines for the following use cases. 

* Audit Data Changes
* Reproduce experiments & reports
* Rollbacks

As you write into a Delta table or directory, every operation is automatically versioned.

You can query by:
1. Using a timestamp
1. Using a version number

using Python, Scala, and/or Scala syntax; for these examples we will use the SQL syntax.  

For more information, refer to [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html)

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Review Delta Lake Table History
All the transactions for this table are stored within this table including the initial set of insertions, update, delete, merge, and inserts with schema modification

In [ ]:
%sql
DESCRIBE HISTORY loan_by_state

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Time Travel via Version Number
Below are SQL syntax examples of Delta Time Travel by using a Version Number

In [ ]:
%sql
SELECT * FROM loan_by_state VERSION AS OF 0

In [ ]:
%sql
SELECT * FROM loan_by_state VERSION AS OF 9

## Run Our Model
Let's run a simple linear regression model to predict the number of loans based on the population of the state
* The following shell statements downloads the us_census_2020 data that we will join with the `loan_by_state` table

In [ ]:
%sh mkdir -p /dbfs/tmp/sais_eu_19_demo/census/ && wget -O /dbfs/tmp/sais_eu_19_demo/census/us_census_2010.csv https://pages.databricks.com/rs/094-YMS-629/images/us_census_2010.csv && ls -al /dbfs/tmp/sais_eu_19_demo/census/

In [ ]:
dbutils.fs.cp("file:/dbfs/tmp/sais_eu_19_demo/census", "dbfs:/tmp/sais_eu_19_demo/census/", recurse=True)

### Notes
If you forgot to install `mlflow` and `yellowbrick` on your cluster, instead of re-running everything all over again:
* Note that the Delta Lake table is stored in `DELTALAKE_SILVERPATH` or `/ml/loan_by_state`
* You can add the libraries, restart the cluster and then start reading the data from the following cells (instead of rerunning everything all over again)
* Just uncomment the cell below to reconnect to your Delta Table

In [ ]:
%pip install mlflow yellowbrick

In [ ]:
# Recreate loan_by_state view
spark.read.format("delta").load("/ml/loan_by_state").createOrReplaceTempView("loan_by_state")
# Check data
display(spark.sql("select count(1) from loan_by_state"))

In [ ]:
# Include census data
census = spark.read.csv('/tmp/sais_eu_19_demo/census/us_census_2010.csv', sep=',', inferSchema=True, header=True)
census.createOrReplaceTempView("census")

# Data versions (0, 6, 9)
dfv0 = spark.sql("select c.Population, l.count as label from (select addr_state as State, count from loan_by_state  version as of 0) l left outer join census c on c.State = l.State")
dfv6 = spark.sql("select c.Population, l.count as label from (select addr_state as State, count from loan_by_state  version as of 6) l left outer join census c on c.State = l.State")
dfv9 = spark.sql("select c.Population, l.count as label from (select addr_state as State, count from loan_by_state  version as of 9 where count is not null) l left outer join census c on c.State = l.State")

In [ ]:
# Calculate predictions
# Initial version of data (v0)
predictLoanCount(dfv0, 'v0')

# Version 6 (after streaming of Iowa data)
displayResiduals(dfv0, dfv6)
predictLoanCount(dfv6, 'v6')

# Version 9 (after correction of data: update, delete, merge)
displayResiduals(dfv0, dfv9)
predictLoanCount(dfv9, 'v9')